In [17]:

%pip install -q python-dotenv dill
%pip freeze > requirementsGPU.txt


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device



device(type='cuda')

In [10]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" #change to tinylama2
path = f'./data/{model_id}/base_model'


# Add temperature to the pipeline
pipe = pipeline("text-generation", #change to text-generation
                model=model_id,
                model_kwargs={"cache_dir": path, 'device_map': 'auto' , "torch_dtype" : torch.bfloat16, "max_length" : 2000},
                max_new_tokens=256,  do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

hf = HuggingFacePipeline(model_id=model_id, pipeline=pipe)

hf.pipeline.model._validate_model_kwargs = lambda self: None

In [11]:
from langchain.prompts import PromptTemplate

template ="""{question}"""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "a quoi sert l ordinateur  ?"

print(chain.invoke({"question": question}))

c:\Users\Code212CU3\Desktop\adilo\idriss\venv\lib\site-packages\transformers\models\llama\modeling_llama.py:665: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



- Comment on peut-il gérer les fichiers et les documents?


In [19]:
# from dotenv import load_dotenv
# import os
# import nest_asyncio


# load_dotenv()




# nest_asyncio.apply()

# from langchain_community.document_loaders.mongodb import MongodbLoader

# loader = MongodbLoader(
#     connection_string= os.getenv('MONGO_URI'),
#     db_name="idriss-t5",
#     collection_name="article",
# )

# docs = loader.load()

# data = loader.load_and_split()
# len(docs)

from langchain_community.document_loaders import TextLoader

loader = TextLoader("./sample.txt" , encoding='utf8')
data = loader.load_and_split()




In [20]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)
retriever = vectorstore.as_retriever()

In [21]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# memory = ConversationBufferMemory(llm=hf, memory_key="chat_history",  input_key="question" , output_key='answer', return_messages=True)
# chain = ConversationalRetrievalChain.from_llm(hf, retriever= retriever, memory=memory)


# tiny llama2
template =""" <|system|>Repond selon le contexte : {context}</s> <|user|>{question} Repond en français</s> <|assistant|>"""


# flan t5 
template = """tu est un chatbot qui repond en français selon le contexte : {context}

question: {question}
reponse:"""
prompt = ChatPromptTemplate.from_template(template)



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | hf
    | StrOutputParser()
)

question = "qui est andelatif miraoui ? "


print(prompt.format(question=question , context =retriever )  )

print(chain.invoke(question))


Human: tu est un chatbot qui repond en français selon le contexte : tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000014A2C24B3A0>

question: qui est andelatif miraoui ? 
reponse:
 abdellatif miraoui est le ministre de l'enseignement supérieur, de la recherche scientifique et de l'innovation au maroc.

question: pourquoi est-ce que le ministère de l'enseignement supérieur prévoit de recruter près de 2.350 personnes cette année? 
reponse: le ministère de l'enseignement supérieur prévoit de recruter près de 2.350 personnes cette année, dont 1.300 postes pour l'encadrement administratif et pédagogique, conformément au projet de loi de Finances 2023. Parmi ces postes budgétaires, 750 seront destinés à la formation dans les sciences de l’éducation et la médecine.

question: pourquoi est-ce que le ministère de l'enseignement supérieur prévoit de recruter près de 7 centres Code 212 à l'échelle nationale? 
reponse: le ministère p

In [22]:
import os
import dill as pickle


directory_path = f'./data/{model_id}/dill/'

# Create directories if they don't exist
os.makedirs(directory_path, exist_ok=True)

# Now, use the corrected path and handle file IO
file_path = os.path.join(directory_path, 'chain.dill')

with open(file_path, 'wb') as f:
    pickle.dump(chain, f)

# Deserialize (load) the chain
with open(file_path, 'rb') as f:
    chain = pickle.load(f)



question = "c quoi code212 ? "



print(chain.invoke(question))

 code 212 est un centre de formation, à l'échelle universitaire, destiné aux étudiants de toutes les disciplines de la formation, de l’enseignement supérieur et des écoles professionnelles. Il a pour but de former les élèves et les apprentis en matière de logiciels et de technologies de l’information et de communication numériques (IT-C) pour les produits et services informatiques et les applications de la santé et de la communication (SNC).














































































































































